In [2]:
!pip install transformers --upgrade
!pip install colpali-engine --upgrade
!pip install qdrant-client[fastembed]

  Using cached transformers-4.49.0-py3-none-any.whl.metadata (44 kB)
Using cached transformers-4.49.0-py3-none-any.whl (10.0 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.1
    Uninstalling transformers-4.47.1:
      Successfully uninstalled transformers-4.47.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
colpali-engine 0.3.8 requires transformers<4.48.0,>=4.47.0, but you have transformers 4.49.0 which is incompatible.
  Using cached transformers-4.47.1-py3-none-any.whl.metadata (44 kB)
Using cached transformers-4.47.1-py3-none-any.whl (10.1 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.49.0
    Uninstalling transformers-4.49.0:
      Successfully uninstalled transformers-4.49.0


In [16]:
from colpali_engine.models import ColPali, ColPaliProcessor, ColQwen2, ColQwen2Processor
from datasets import load_dataset 
from qdrant_client import QdrantClient, models
import torch
from tqdm import tqdm
import uuid
import numpy as np
import random
import os
from PIL import Image

In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("qdrant_api")


In [7]:
from qdrant_client import QdrantClient
client = QdrantClient( url='https://d3ceafa0-b886-47c4-aa87-c4e9c91905d6.eu-central-1-0.aws.cloud.qdrant.io',api_key = secret_value_0)
print(client.get_collections())

collections=[CollectionDescription(name='16_dang_bai_tap')]


In [6]:
colpali_model = ColPali.from_pretrained(
        "vidore/colpali-v1.2",
        torch_dtype=torch.bfloat16,
        device_map="cuda:0",  # Use "cuda:0" for GPU, "cpu" for CPU, or "mps" for Apple Silicon
    ).eval()

colpali_processor = ColPaliProcessor.from_pretrained("vidore/colpali-v1.2")

adapter_config.json:   0%|          | 0.00/750 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/66.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/862M [00:00<?, ?B/s]

...

adapter_model.safetensors:   0%|          | 0.00/78.6M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/243k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/733 [00:00<?, ?B/s]

In [8]:
collection_name = '16_dang_bai_tap'
client.delete_collection(collection_name=f"{collection_name}")

True

In [ ]:
collection_name = 'hinhkhonggian'
client.delete_collection(collection_name=f"{collection_name}")

In [9]:
collection_name = '50_cau_hoi_va_loi_giai'
client.create_collection(
    collection_name=collection_name,
    vectors_config={
        "original": 
            models.VectorParams( #switch off HNSW
                  size=128,
                  distance=models.Distance.COSINE,
                  multivector_config=models.MultiVectorConfig(
                      comparator=models.MultiVectorComparator.MAX_SIM
                  ),
                  hnsw_config=models.HnswConfigDiff(
                      m=0 #switching off HNSW
                  )
            ),
        "mean_pooling_columns": models.VectorParams(
                size=128,
                distance=models.Distance.COSINE,
                multivector_config=models.MultiVectorConfig(
                    comparator=models.MultiVectorComparator.MAX_SIM
                )
            ),
        "mean_pooling_rows": models.VectorParams(
                size=128,
                distance=models.Distance.COSINE,
                multivector_config=models.MultiVectorConfig(
                    comparator=models.MultiVectorComparator.MAX_SIM
                )
            )
    }
)

True

In [10]:
def get_patches(image_size, model_processor, model, model_name):
    if model_name == "colPali":
        return model_processor.get_n_patches(image_size, 
                                             patch_size=model.patch_size)
    elif model_name == "colQwen":
        return model_processor.get_n_patches(image_size, 
                                             patch_size=model.patch_size,
                                             spatial_merge_size=model.spatial_merge_size)
    return None, None

In [11]:
def embed_and_mean_pool_batch(image_batch, model_processor, model, model_name):
    #embed
    with torch.no_grad():
        processed_images = model_processor.process_images(image_batch).to(model.device) 
        image_embeddings = model(**processed_images)

    image_embeddings_batch = image_embeddings.cpu().float().numpy().tolist()
    
    #mean pooling
    pooled_by_rows_batch = []
    pooled_by_columns_batch = []
    
    
    for image_embedding, tokenized_image, image in zip(image_embeddings, 
                                                       processed_images.input_ids, 
                                                       image_batch):
        x_patches, y_patches = get_patches(image.size, model_processor, model, model_name)
        #print(f"{model_name} model divided this PDF page in {x_patches} rows and {y_patches} columns")

        image_tokens_mask = (tokenized_image == model_processor.image_token_id)
        
        image_tokens = image_embedding[image_tokens_mask].view(x_patches, y_patches, model.dim)
        pooled_by_rows = torch.mean(image_tokens, dim=0)
        pooled_by_columns = torch.mean(image_tokens, dim=1)

        image_token_idxs = torch.nonzero(image_tokens_mask.int(), as_tuple=False)
        first_image_token_idx = image_token_idxs[0].cpu().item()
        last_image_token_idx = image_token_idxs[-1].cpu().item()
        
        prefix_tokens = image_embedding[:first_image_token_idx]
        postfix_tokens = image_embedding[last_image_token_idx + 1:]
        
        #print(f"There are {len(prefix_tokens)} prefix tokens and {len(postfix_tokens)} in a {model_name} PDF page embedding")

        #adding back prefix and postfix special tokens
        pooled_by_rows = torch.cat((prefix_tokens, pooled_by_rows, postfix_tokens), dim=0).cpu().float().numpy().tolist()
        pooled_by_columns = torch.cat((prefix_tokens, pooled_by_columns, postfix_tokens), dim=0).cpu().float().numpy().tolist()
        
        pooled_by_rows_batch.append(pooled_by_rows)
        pooled_by_columns_batch.append(pooled_by_columns)


    return image_embeddings_batch, pooled_by_rows_batch, pooled_by_columns_batch

In [12]:
def upload_batch(client, original_batch, pooled_by_rows_batch, pooled_by_columns_batch, payload_batch, collection_name):
    try:
        client.upload_collection(
            collection_name=collection_name,
            vectors={
                "mean_pooling_columns": pooled_by_columns_batch,
                "original": original_batch,
                "mean_pooling_rows": pooled_by_rows_batch
            },
            payload=payload_batch,
            ids=[str(uuid.uuid4()) for i in range(len(original_batch))]
        )
    except Exception as e:
        print(f"Error during upsert: {e}")

## Create dataset


In [11]:
!sudo apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpoppler-dev libpoppler-private-dev libpoppler118
The following NEW packages will be installed:
  poppler-utils
The following packages will be upgraded:
  libpoppler-dev libpoppler-private-dev libpoppler118
3 upgraded, 1 newly installed, 0 to remove and 126 not upgraded.
Need to get 1,462 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpoppler-private-dev amd64 22.02.0-2ubuntu0.6 [199 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpoppler-dev amd64 22.02.0-2ubuntu0.6 [5,184 B]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpoppler118 amd64 22.02.0-2ubuntu0.6 [1,071 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.6 [186 kB]
Fetched 1,4

In [ ]:
import os
from tqdm import tqdm
from PIL import Image
from pdf2image import convert_from_path
# Path to the PDF file
pdf_path = 'C:\Users\huyho\OneDrive\Desktop\DAT_MathRag\combined_output_12.pdf'

# Folder to save images
output_folder = '/kaggle/working/images'

# Create the folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Convert PDF pages to images
pages = convert_from_path(pdf_path)

# Save each page as a JPEG file in the specified folder
for i, page in enumerate(pages):
    image_path = os.path.join(output_folder, f'page_{i+1}.jpg')
    page.save(image_path, 'JPEG')

In [3]:
import fitz  # PyMuPDF
from PIL import Image

def pdf_to_images(pdf_path, output_folder):
    # Open the PDF file
    doc = fitz.open(pdf_path)
    
    for page_num in range(len(doc)):
        page = doc[page_num]  # Get a page
        pix = page.get_pixmap()  # Render page as an image
        
        # Convert to PIL image (optional)
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        
        # Save the image
        image_path = f"{output_folder}/page_{page_num + 1}.png"
        img.save(image_path, "PNG")
        print(f"Saved: {image_path}")

# Example usage
pdf_to_images(r"C:\Users\huyho\OneDrive\Desktop\DAT_MathRag\combined_output_12.pdf", "50_cau_hoi")

Saved: 50_cau_hoi/page_1.png
Saved: 50_cau_hoi/page_2.png
Saved: 50_cau_hoi/page_3.png
Saved: 50_cau_hoi/page_4.png
Saved: 50_cau_hoi/page_5.png
Saved: 50_cau_hoi/page_6.png
Saved: 50_cau_hoi/page_7.png
Saved: 50_cau_hoi/page_8.png
Saved: 50_cau_hoi/page_9.png
Saved: 50_cau_hoi/page_10.png
Saved: 50_cau_hoi/page_11.png
Saved: 50_cau_hoi/page_12.png
Saved: 50_cau_hoi/page_13.png
Saved: 50_cau_hoi/page_14.png
Saved: 50_cau_hoi/page_15.png
Saved: 50_cau_hoi/page_16.png
Saved: 50_cau_hoi/page_17.png
Saved: 50_cau_hoi/page_18.png
Saved: 50_cau_hoi/page_19.png
Saved: 50_cau_hoi/page_20.png
Saved: 50_cau_hoi/page_21.png
Saved: 50_cau_hoi/page_22.png
Saved: 50_cau_hoi/page_23.png
Saved: 50_cau_hoi/page_24.png
Saved: 50_cau_hoi/page_25.png
Saved: 50_cau_hoi/page_26.png
Saved: 50_cau_hoi/page_27.png
Saved: 50_cau_hoi/page_28.png
Saved: 50_cau_hoi/page_29.png
Saved: 50_cau_hoi/page_30.png
Saved: 50_cau_hoi/page_31.png
Saved: 50_cau_hoi/page_32.png
Saved: 50_cau_hoi/page_33.png
Saved: 50_cau_hoi/p

In [14]:
import os

In [26]:
base_directory = '/kaggle/input/50-question-and-answer/50_cau_hoi'

image_url = os.listdir(base_directory)

# image_url = list(map(lambda item: f"{item}", image_url))
from pandas import DataFrame

payloads = DataFrame.from_records({'image_url':image_url})

images = list(map(lambda el: Image.open(os.path.join(base_directory,el)).convert('RGB'), payloads["image_url"]))


In [27]:
len(images)

50

In [28]:
import pandas as pd

image_dir = pd.DataFrame({'image':images, 'image_url':image_url})
image_dir.head()

,image,image_url
0,<PIL.Image.Image image mode=RGB size=1161x662 ...,page_40.png
1,<PIL.Image.Image image mode=RGB size=928x513 a...,page_39.png
2,<PIL.Image.Image image mode=RGB size=675x116 a...,page_20.png
3,<PIL.Image.Image image mode=RGB size=752x237 a...,page_35.png
4,<PIL.Image.Image image mode=RGB size=712x181 a...,page_3.png


In [29]:
batch_size = 1 #based on available compute
# dataset_source = pdf_path
collection_name = "50_cau_hoi_va_loi_giai"

with tqdm(total=len(images), desc=f"Uploading progress of dataset to \"{collection_name}\" collection") as pbar:
    for i in range(0, len(image_dir), batch_size):
        batch = image_dir[i : i + batch_size]
        image_batch = batch['image']
        current_batch_size = len(image_batch)
        try:
            original_batch, pooled_by_rows_batch, pooled_by_columns_batch = embed_and_mean_pool_batch(image_batch, 
                                                                                          colpali_processor, 
                                                                                          colpali_model, 
                                                                                          "colPali")
        except Exception as e:
            print(f"Error during embed: {e}")
            continue
        try:
            upload_batch(
                client,
                np.asarray(original_batch, dtype=np.float32),
                np.asarray(pooled_by_rows_batch, dtype=np.float32),
                np.asarray(pooled_by_columns_batch, dtype=np.float32),
                [
                    {
                        "source": image_url[i], 
                        "index": j
                    }
                    for j in range(i, i + current_batch_size)
                ],
                collection_name
            )
        except Exception as e:
            print(f"Error during upsert: {e}")
            continue
        # Update the progress bar
        pbar.update(current_batch_size)
print("Uploading complete!")

Uploading progress of dataset to "50_cau_hoi_va_loi_giai" collection: 100%|██████████| 50/50 [03:40<00:00,  4.41s/it]

Uploading complete!


In [30]:
def batch_embed_query(query_batch, model_processor, model):
    with torch.no_grad():
        processed_queries = model_processor.process_queries(query_batch).to(model.device)
        query_embeddings_batch = model(**processed_queries)
    return query_embeddings_batch.cpu().float().numpy()

In [31]:
def batch_embed_query_image(query_batch, model_processor, model):
    with torch.no_grad():
        processed_images = model_processor.process_images(query_batch).to(model.device)
        image_embeddings_batch = model(**processed_images)
    return image_embeddings_batch.cpu().float().numpy()

In [32]:
query_path = '/kaggle/input/hinh-khong-gian-images/pdf_hinh_khong_gian/page_1.png'
query_image = Image.open(query_path).convert('RGB')

colpali_query = batch_embed_query_image([query_image], colpali_processor, colpali_model)

In [30]:
query = "Góc giữa đường thẳng và mặt phẳng"

colpali_query = batch_embed_query([query], colpali_processor, colpali_model)
print(f"ColPali embedded query \"{query}\" with {len(colpali_query[0])} multivectors of dim {len(colpali_query[0][0])}")

ColPali embedded query "Góc giữa đường thẳng và mặt phẳng" with 23 multivectors of dim 128


In [34]:
def reranking_search_batch(client,
                           query_batch,
                           collection_name,
                           search_limit=20,
                           prefetch_limit=200):
    search_queries = [
      models.QueryRequest(
          query=query,
          prefetch=[
              models.Prefetch(
                  query=query,
                  limit=prefetch_limit,
                  using="mean_pooling_columns"
              ),
              models.Prefetch(
                  query=query,
                  limit=prefetch_limit,
                  using="mean_pooling_rows"
              ),
          ],
          limit=search_limit,
          with_payload=True,
          with_vector=False,
          using="original"
      ) for query in query_batch
    ]
    return client.query_batch_points(
        collection_name=collection_name,
        requests=search_queries
    )

In [35]:
answer_colpali = reranking_search_batch(client, colpali_query, collection_name)

In [36]:
answer_colpali[0].points[0].payload['source']

'page_34.png'

In [8]:
from PIL import Image
import base64
import io

In [3]:
query_path = r'C:\Users\huyho\OneDrive\Desktop\DAT_MathRag\16_dang_bai_image\page_1.png'
query_image = Image.open(query_path).convert('RGB')

In [6]:
def pil_to_base64(image):
    """Convert a PIL image to a base64 string."""
    buffered = io.BytesIO()
    image.save(buffered, format="PNG")  # Change format if needed (JPEG, etc.)
    return base64.b64encode(buffered.getvalue()).decode("utf-8")

In [9]:
bs64 = pil_to_base64(query_image)

In [10]:
bs64

'...'